In [2]:
# python script of M4_weekly and DeepAR

# standard imports
import numpy as np
import pandas as pd

# json
import json

# gluon data
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator


# model imports
# from gluonts.model.deepstate import DeepStateEstimator
# from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint

# seeds
mx.random.seed(42)
np.random.seed(42)

if __name__ == "__main__":

    dataset = get_dataset("m4_yearly", regenerate=False)

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=1,      # default: 100
        num_batches_per_epoch=50,      #default: 50
        learning_rate=1e-3,
        # hybridize=False,
    )

    # cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        # context_length=168,
        # cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
    )

    predictor = estimator.train(dataset.train)


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_yearly.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 13463
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 16.81it/s, avg_epoch_loss=8.44]
INFO:root:Epoch[0] Elapsed time 3.004 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=8.441840
INFO:root:Loading parameters from best epoch (0)
INFO:root:Final loss: 8.44183964729309 (occurred at epoch 0)
INFO:root:End model training


In [3]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
)


In [6]:
list(forecast_it)

KeyboardInterrupt: 

In [4]:
agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
)


Running evaluation:   0%|          | 0/23000 [00:00<?, ?it/s]


AssertionError: Cannot extract prediction target since the index of forecast is outside the index of target
Index of forecast: DatetimeIndex(['1752-07-31', '1753-07-31', '1754-07-31', '1755-07-31',
               '1756-07-31', '1757-07-31'],
              dtype='datetime64[ns]', freq='12M')
 Index of target: DatetimeIndex(['1749-12-31', '1750-12-31', '1751-12-31', '1752-12-31',
               '1753-12-31', '1754-12-31', '1755-12-31', '1756-12-31',
               '1757-12-31', '1758-12-31', '1759-12-31', '1760-12-31',
               '1761-12-31', '1762-12-31', '1763-12-31', '1764-12-31',
               '1765-12-31', '1766-12-31', '1767-12-31', '1768-12-31',
               '1769-12-31', '1770-12-31', '1771-12-31', '1772-12-31',
               '1773-12-31', '1774-12-31', '1775-12-31', '1776-12-31',
               '1777-12-31', '1778-12-31', '1779-12-31', '1780-12-31',
               '1781-12-31', '1782-12-31', '1783-12-31', '1784-12-31',
               '1785-12-31'],
              dtype='

In [ ]:
pprint(agg_metrics)